In [1]:
!pip install gspread



[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install gspread-dataframe


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install pandas


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install snowflake-connector-python


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine
from snowflake.connector.pandas_tools import pd_writer
import time 
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

In [7]:
start_time = time.time()

engine = create_engine(URL(
                    account ='xsymtcm-em56590',
                    user ='amritaneogi',
                    password ='ToblueBear@17',
                    database ='HOUSE_PRICE',
                    schema ='PUBLIC',
                    warehouse ='PRICE_WH'))
                
with engine.connect() as conn:
    try:
        query = """ SELECT ID, TITLE FROM OTODOM_DATA_FLATTEN ORDER BY ID limit 300"""

        df = pd.read_sql(query,conn) # reading data-- 300 records
        
        
        # making connection between this sript and google drive
        gc = gspread.service_account(filename='C:/Users/amrit/OneDrive/Documents/GitHub/Data_Analytics_Project-Housing_Price_Profiler/gspread/majestic-bounty-402621-929a2d689e3c.json')
        

        loop_counter = 0
        chunk_size = 100 # google sheet to be created for every 100 records; we have 300 records so 3 files to be created
        file_name = 'OTODOM_ANALYSIS_'  # google sheet will be created with this name
        user_email = 'amritaneogi0111@gmail.com'

        for i in range(0,len(df),chunk_size):
            loop_counter += 1
            df_in = df.iloc[i:(i+chunk_size), :]

            spreadsheet_title = file_name + str(loop_counter)
            try:
                locals()['sh'+str(loop_counter)] = gc.open(spreadsheet_title)
            except gspread.SpreadsheetNotFound:
                locals()['sh'+str(loop_counter)] = gc.create(spreadsheet_title)

            locals()['sh'+str(loop_counter)].share(user_email, perm_type='user', role='writer')
            wks = locals()['sh'+str(loop_counter)].get_worksheet(0)
            wks.resize(len(df_in)+1) # resizing because by default google sheets create 1000 records, but here I want only 100
            set_with_dataframe(wks, df_in)  # loading data from pandas dataframe to the worksheet
                
            column = 'C'   # Column to apply the formula 
            start_row = 2  # Starting row to apply the formula
            end_row = wks.row_count   # Ending row to apply the formula
            cell_range = f'{column}{start_row}:{column}{end_row}' 
            curr_row = start_row
            cell_list = wks.range(cell_range) # returns all the cells where formula needs to be applied
            
            for cell in cell_list:
                cell.value = f'=GOOGLETRANSLATE(B{curr_row},"pl","en")'
                curr_row += 1
                
            # Update the worksheet with the modified cells
            wks.update_cells(cell_list, value_input_option='USER_ENTERED')

            print(f'Spreadsheet {spreadsheet_title} created!')

            # creating a log table
            # it takes 30 to 60 mins to get all the formulae applied
            # so I do not want to immediately do all the transformation in google sheet and load it back to snowflake, rather wait for some time for the formulae to update completely
            df_log = pd.DataFrame({'ID':[loop_counter], 'SPREADSHEET_NAME':[spreadsheet_title]})
            
            # load the spreadsheet to a log table, once all the formulae is updated, run another script to load data from google sheet back to Python and then to snowflake
            df_log.to_sql('OTODOM_DATA_LOG', con=engine, if_exists='append', index=False)

    except Exception as e:
        print('--- Error --- ',e)
    finally:
        conn.close()
engine.dispose()

print("--- %s seconds ---" % (time.time() - start_time))

Spreadsheet OTODOM_ANALYSIS_1 created!


C:\Users\amrit\anaconda3\lib\site-packages\pandas\io\sql.py:1685: UserWarning: The provided table name 'OTODOM_DATA_LOG' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


Spreadsheet OTODOM_ANALYSIS_2 created!
Spreadsheet OTODOM_ANALYSIS_3 created!
--- 26.97686505317688 seconds ---
